In [17]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import tensorflow as tf

In [18]:
#Importando dataset Acelerômetro
caminho = "SmartWatch/accel/"
arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith('.txt')]
acel = pd.DataFrame()
for i in arquivos:
  acc = pd.read_csv(caminho+i, header=None, sep=",")
  acel = pd.concat([acel, acc], ignore_index=True)
acel[5] = acel[5].replace(';', '', regex=True)
acel.rename(columns={0: "pessoa", 1: "atividade", 2: "tempo", 3: "acelerômetro_X", 4:"acelerômetro_Y", 5:"acelerômetro_Z"}, inplace=True)
acel


,pessoa,atividade,tempo,acelerômetro_X,acelerômetro_Y,acelerômetro_Z
0,1600,A,90426708196641,7.091625,-0.591667,8.195502
1,1600,A,90426757696641,4.972757,-0.158317,6.6967316
2,1600,A,90426807196641,3.253720,-0.191835,6.107758
3,1600,A,90426856696641,2.801216,-0.155922,5.997625
4,1600,A,90426906196641,3.770868,-1.051354,7.731027
...,...,...,...,...,...,...
3777041,1650,S,2426184421424592,1.486500,-13.302334,5.355231
3777042,1650,S,2426184471411932,4.017171,-13.146711,5.1182046
3777043,1650,S,2426184521399272,4.412214,-15.318253,3.3704374
3777044,1650,S,2426184571385262,4.426579,-13.012636,2.5755625


In [19]:
#Importando dataset Giroscópio
caminho = "SmartWatch/gyro/"
arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith('.txt')]
giro = pd.DataFrame()
for i in arquivos:
  g = pd.read_csv(caminho+i, header=None, sep=",")
  giro = pd.concat([giro, g], ignore_index=True)
giro[5] = giro[5].replace(';', '', regex=True)
giro.rename(columns={0: "pessoa", 1: "atividade", 2: "tempo", 3: "giroscópio_X", 4:"giroscópio_Y", 5:"giroscópio_Z"}, inplace=True)
giro

,pessoa,atividade,tempo,giroscópio_X,giroscópio_Y,giroscópio_Z
0,1600,A,90426757696641,0.314944,-1.022277,-0.3099616
1,1600,A,90426807196641,0.387382,-0.618541,-0.048971802
2,1600,A,90426856696641,0.070999,-0.209480,-0.1959783
3,1600,A,90426906196641,0.037975,0.254976,-0.1565635
4,1600,A,90426955696641,0.073129,0.719431,-0.0010349044
...,...,...,...,...,...,...
3440337,1650,S,2426184371437252,-2.856065,2.057101,0.85393304
3440338,1650,S,2426184421424592,-2.911458,1.817417,1.2811041
3440339,1650,S,2426184471411932,-3.015854,1.550036,2.0576818
3440340,1650,S,2426184521399272,-3.218254,1.595842,3.0185504


In [20]:
acel["acelerômetro_Z"] = pd.to_numeric(acel["acelerômetro_Z"], errors='coerce')
giro["giroscópio_Z"] = pd.to_numeric(giro["giroscópio_Z"], errors='coerce')

In [21]:
dataset = pd.merge(acel, giro, on=['tempo', 'atividade'])
dataset.drop(columns=['pessoa_y'],inplace=True)
dataset

,pessoa_x,atividade,tempo,acelerômetro_X,acelerômetro_Y,acelerômetro_Z,giroscópio_X,giroscópio_Y,giroscópio_Z
0,1600,A,90426757696641,4.972757,-0.158317,6.696732,0.314944,-1.022277,-0.309962
1,1600,A,90426807196641,3.253720,-0.191835,6.107758,0.387382,-0.618541,-0.048972
2,1600,A,90426856696641,2.801216,-0.155922,5.997625,0.070999,-0.209480,-0.195978
3,1600,A,90426906196641,3.770868,-1.051354,7.731027,0.037975,0.254976,-0.156563
4,1600,A,90426955696641,4.661511,0.169689,9.684695,0.073129,0.719431,-0.001035
...,...,...,...,...,...,...,...,...,...
3368537,1650,S,2426184371437252,2.137723,-7.211485,8.065467,-2.856065,2.057101,0.853933
3368538,1650,S,2426184421424592,1.486500,-13.302334,5.355231,-2.911458,1.817417,1.281104
3368539,1650,S,2426184471411932,4.017171,-13.146711,5.118205,-3.015854,1.550036,2.057682
3368540,1650,S,2426184521399272,4.412214,-15.318253,3.370437,-3.218254,1.595842,3.018550


In [22]:
featuresX = ['acelerômetro_X', 'acelerômetro_Y', 'acelerômetro_Z', 'giroscópio_X', 'giroscópio_Y', 'giroscópio_Z']

def normaliza(coluna):
    media = coluna.mean()
    desvio_padrao = coluna.std()
    normal = (coluna - media)/desvio_padrao
    return normal

In [23]:
for i in featuresX:
    dataset[i] = normaliza(dataset[i])

In [24]:
dataset = dataset.drop_duplicates()

In [25]:
letras = ['A','B','C','D','E','F','G','H','I','J','K','L','M','O','P','Q','R','S']
for i in range(18):
    dataset.replace({letras[i]:i}, inplace=True)
dataset["atividade"] = pd.to_numeric(dataset["atividade"], errors='coerce')
dataset = pd.get_dummies(dataset, columns=['atividade'])

C:\Users\rapha\AppData\Local\Temp\ipykernel_9456\3298386683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.replace({letras[i]:i}, inplace=True)
C:\Users\rapha\AppData\Local\Temp\ipykernel_9456\3298386683.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["atividade"] = pd.to_numeric(dataset["atividade"], errors='coerce')


In [26]:
dataset.to_csv('normalizado.csv', index=False)